- author: Lee Meng
- date: 2019-09-21 17:22
- title: 透過 Manim 動畫直觀理解神經網路 & 線性代數
- slug: understand-neural-net-and-linear-algebra-using-manim
- tags: Manim, Python
- summary: s
- description: s 
- image: TwoLayersReLUInBetweenSolveHardTwoCurves.jpg

!quote
- 這是篇透過動畫來「直觀」理解神經網路的科普文。讀完本文，你將能夠更直觀地體會神經網路以及其與線性代數之間的緊密關係，並進一步展開自己的深度學習之旅。

（小提醒：因動畫皆為暗色背景，本文適合以**暗色模式**閱讀。你可以點擊頁面左下的按鈕來切換模式）

這是個眾人對人工智慧（**A**rtificial **I**ntelligence, AI）趨之若鶩的時代。此領域近年的快速發展很大一部份可歸功於[深度學習](https://leemeng.tw/deep-learning-resources.html)以及[人工神經網路（**N**eural **N**etwork, 後簡稱為 NN）](https://zh.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C)。深度學習框架也日漸成熟，讓任何人都可以輕易地使用 [TensorFlow](https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/beginner.ipynb) 或 [PyTorch](https://pytorch.org/) 在 30 秒內就訓練出一個具有 98% 正確率的阿拉伯數字辨識模型：

```python
# 此例使用 TensorFlow，但各大深度學習框架的實現邏輯基本上類似
import tensorflow as tf

# 載入深度學習 Hello World: MNIST 數字 dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 建立一個具有 10 萬參數的神經網路（模型）
# 在現在模型參數動輒上千萬的年代，此神經網路可說是小到不行
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# 選擇 loss func、optimizer 並訓練模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

# 訓練後的 NN 在測試集上可得到近 98% 正確辨識率
model.evaluate(x_test, y_test)

# 測試結果
# loss: 0.0750 - accuracy: 0.9763
```

你也可以使用[這個 Colab 筆記本](https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/beginner.ipynb)自己訓練看看。我假設你已經能夠讀懂上面這段 [Python](https://www.python.org/) 程式碼並了解[最基本的神經網路概念](https://www.youtube.com/watch?v=Dr-WRlEFefw)。本文的目的是透過一些圖片與動畫，來讓你對神經網路（NN）多些更**直觀**的理解。

回到我們剛剛建立的 NN。儘管擁有近 10 萬個可訓練參數，上面這個透過 [Keras](https://www.tensorflow.org/guide/keras) 定義的 `model` 是個在深度學習領域裡會被視為 Hello World 等級的**簡單**神經網路。這很正常，畢竟我們在最近 [BERT](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html) 以及 [GPT-2 文章](https://leemeng.tw/gpt2-language-model-generate-chinese-jing-yong-novels.html)裡看到的都是擁有**上億**參數的強大神經網路。

!image
- manim/mnist-simple.jpg
- 輸入是 28*28 = 768 維的圖片像素，輸出則是 10 個數字類別的簡單 2-layers NN

但就算是一個如此「簡單」的 NN，我相信仍有不少人「見山不是山」。

講白點，儘管會使用 TensorFlow 或是 PyTorch 來建立一個這樣的 NN 的人很多，不少人（包含剛入門的我）事實上並不對神經網路有足夠直觀的理解。而之所以會有這樣的現象，主要是因為：
- 強大的深度學習框架把底層細節邏輯都包了起來
- 不少人還沒搞懂跟 NN 相關的[線性代數](https://www.youtube.com/watch?v=fNk_zzaMoSs&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)概念
- 能視覺化神經網路內部機制的手法並不那麼普遍

為了讓更多人能夠更**直觀**地體會神經網路的運作方式，我將透過以下 3 者：
1. 繪圖工具：[3Blue1Brown](https://www.youtube.com/channel/UCYO_jab_esuFRV4b17AJtAw) 的強大 Python 動畫引擎 [Manim](https://github.com/3b1b/manim)
2. 學習任務：比 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) 還簡單的[二分類（Binary Classification）](https://en.wikipedia.org/wiki/Binary_classification)問題
3. 模型架構：1 到 2 層、只有不到 10 個參數的超級簡單神經網路

來闡述基本的 NN 以及相關的線性代數（Linear Algebra）概念。如果這 idea 聽起來還不錯，或是你喜歡看動畫來了解數學概念，那我會建議你繼續閱讀：）

## 第一印象：直觀感受簡單 NN 如何解決二分類任務

你的時間寶貴，因此我事先將本文的精華都濃縮在底下這個 1 分鐘的短片了。此影片展示了一個簡單神經網路解決二分類任務的**完整過程**。我等等會更仔細地說明影片內容，但現在請你馬上點擊播放鍵觀看吧！

!mp4
- options: no-loop, no-autoplay, controls
- images/manim/TwoLayersReLUInBetweenSolveHardTwoCurves.mp4

!quote
- 這是我自己看過最簡單、同時又最美麗的神經網路。希望你也能感同身受。

我不知道看影片前的你對神經網路的理解程度，但我相信很多人都能在這短短的一分鐘內（重新）獲得些新的啟發。如果你願意，我建議你再看一次影片並在需要時暫停影片咀嚼。影片裡有很多基本、但十分重要的概念值得掌握。

你有再看一遍嗎？讓我幫你把影片裡想要傳達的概念一一列出：
- dd
